In [1]:
import KeyFunctions as kf
import pandas as pd
import numpy as np
import datetime

root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

In [2]:
#Import the postcode list from the raw data gdf
gdf = kf.load_obj(root_path,"raw_gdf")
# name = "StPauls" 
# name = "Grasmere"
# gdf = kf.load_obj(root_path,"raw_gdf_" + name)

In [ ]:
gdf["Type"].unique().tolist()

['RailwayStations',
 'FireStations',
 'Crimes',
 'StopAndSearch',
 'Clinic',
 'Dentists',
 'GP',
 'Hospital',
 'Opticians',
 'Pharmacy',
 'Care homes and care at home',
 'All_GB',
 'National_Parks',
 'LocalAuthorities',
 'LSOA',
 'GreenSpace',
 'Rivers',
 'Railway_Lines',
 'Woodland_Region',
 'Urban_Region',
 'Foreshor_Region',
 'Ferry_Line',
 'Coastline',
 'Lakes',
 'Road',
 'MotorwayJunction',
 'Peat',
 'SiteOfScientificInterest',
 'LocalNatureReserve',
 'NationalNatureReserve',
 'SpecialAreaOfConservation',
 'AncientWoodland',
 'ConservationArea',
 'Listed Buildings',
 'MetOffice']

In [ ]:
#Shapes
LSOA = gdf.loc[gdf["Type"]=="LSOA",:].reset_index(drop=True)

#Points
LReg = gdf.loc[gdf["Type"]=="LReg",:].reset_index(drop=True)
Crimes = gdf.loc[gdf["Type"]=="Crimes",:].reset_index(drop=True)
UDPRNs = gdf.loc[gdf["Type"]=="UDPRNs",:].reset_index(drop=True)

In [ ]:
#Add Sales and Average Cost for each LSOA
LSOA["LReg_Sales_In_LSOA"] = [np.count_nonzero(LReg.loc[LReg.within(poly_geom),"Details_Float"].to_numpy()) for poly_geom in LSOA.geometry]

In [ ]:
LSOA["LReg_AvCost_In_LSOA"] = [np.sum(LReg.loc[LReg.within(poly_geom),"Details_Float"].to_numpy()) for poly_geom in LSOA.geometry]/LSOA["LReg_Sales_In_LSOA"]

KeyboardInterrupt: 

In [ ]:
LSOA["UDPRNs"] = [np.count_nonzero(UDPRNs.loc[UDPRNs.within(poly_geom),"Details_Float"].to_numpy()) for poly_geom in LSOA.geometry]

In [ ]:
#Add crimes for each LSOA
Crime_Types = set(gdf.loc[gdf["Type"]=="Crimes",:].loc[:,"Name"].to_list())
Crime_List = []

for c in Crime_Types:
    print(c)
    new_name = "Crimes_per1000houses_"+c.replace(' ', '_')
    sub_Crimes = Crimes.loc[Crimes["Name"]==c,:]
    LSOA[new_name] = ([np.count_nonzero(sub_Crimes.loc[sub_Crimes.within(poly_geom),"Details_Float"].to_numpy()) for poly_geom in LSOA.geometry] / LSOA["UDPRNs"])*1000
    Crime_List = Crime_List + [new_name]

Possession of weapons
Vehicle crime
Drugs
Other crime
Shoplifting
Other theft
Theft from the person
Anti-social behaviour
Public order
Bicycle theft
Criminal damage and arson
Robbery
Burglary
Violence and sexual offences


In [ ]:
#Create LSOA Network
network, pos = kf.create_network_from_shapes(LSOA)

#Calculate densities
LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "LReg_Sales_In_LSOA", "LReg_AvCost_In_LSOA", 1).iloc[:,1]], axis=1)
LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "LReg_Sales_In_LSOA", "LReg_AvCost_In_LSOA", 3).iloc[:,1]], axis=1)
LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "LReg_Sales_In_LSOA", "LReg_AvCost_In_LSOA", 5).iloc[:,1]], axis=1)

d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = np.round(np.sum(new[count] * new[average])/ sales)
d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = np.round(np.sum(new[count] * new[average])/ sales)
d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = np.round(np.sum(new[count] * new[average])/ sales)
d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = np.round(np.sum(new[count] * new[average])/ sales)
d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = np.round(np.sum(new[count] * new[average])/ sales)
d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = np.round(np.sum(new[count] * new[aver

In [ ]:
for c in Crime_List:
    LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "UDPRNs", c, 1).iloc[:,1]], axis=1)
    LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "UDPRNs", c, 2).iloc[:,1]], axis=1)
    LSOA = pd.concat([LSOA, kf.average_within_n_boundaries(LSOA, "UDPRNs", c, 5).iloc[:,1]], axis=1)

LSOA = LSOA.drop(columns=['Type', 'Details_Str', 'Details_Float', 'geometry'])

del LReg, Crimes, UDPRNs

c:\Users\richa\.conda\envs\GeoData\Lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)
d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = np.round(np.sum(new[count] * new[average])/ sales)
d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = np.round(np.sum(new[count] * new[average])/ sales)
d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = np.round(np.sum(new[count] * new[average])/ sales)
d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = np.round(np.sum(new[count] * new[average])/ sales)
d:\G_Drive\Coding\GeoData\KeyFunctions.py:289: RuntimeWarning: invalid value encountered in scalar divide
  Av_Cost = 

In [ ]:
#get nspl data
NSPL_infile = root_path + "NSPL_NOV_2020_UK/Data/NSPL_NOV_2020_UK.csv"

NSPL = pd.read_csv(NSPL_infile, low_memory=False)

LSOA_Lookup_path = root_path + "NSPL_NOV_2020_UK/Documents/LSOA (2011) names and codes UK as at 12_12.csv"

LSOA_Lookup = pd.read_csv(LSOA_Lookup_path)

In [ ]:
#Combine the NSPL data with it's lookups for the variables we want
NSPL = NSPL.loc[:,["pcd", "lsoa11"]]
NSPL["pcd"] = NSPL["pcd"].astype(str).str.replace(" ","")
NSPL = NSPL.merge(LSOA_Lookup, left_on="lsoa11", right_on="LSOA11CD")
NSPL = NSPL.drop(["lsoa11", "LSOA11CD"], axis=1)
NSPL = NSPL.rename(columns={"pcd": "Postcode"})
NSPL = NSPL.rename(columns={"LSOA11NM": "LSOA"})

In [ ]:
#Initialise our postcodes gdf as a list of postcodes and their points
Postcodes = gdf.loc[gdf["Type"]=="Postcodes",["Name", "geometry"]].copy()
Postcodes = Postcodes.rename(columns={"Name": "Postcode"})

Postcodes = Postcodes.set_index("Postcode")

#Add on nspl data
Postcodes = Postcodes.merge(NSPL, how="left", left_on="Postcode", right_on="Postcode")
Postcodes = Postcodes.merge(LSOA, left_on="LSOA", right_on="Name",how="inner")

In [ ]:
#List out the points we will want to use for densities, point distances...etc.

All_Crimes = gdf.loc[gdf["Type"]=="Crimes",:]
All_Schools = gdf.loc[gdf["Type"]=="Schools",:]
Hospitals = gdf.loc[gdf["Type"]=="Hospital",:]
RailwayStations = gdf.loc[gdf["Type"]=="RailwayStations",:]
FerryTerminals = gdf.loc[gdf["Type"]=="FerryTerminals",:]
BusStops = gdf.loc[gdf["Type"]=="BusStops",:]
ListedBuildings = gdf.loc[gdf["Type"]=="Listed Buildings",:]
PropertySales = gdf.loc[gdf["Type"]=="LReg",:]
RoadAccidents = gdf.loc[gdf["Type"]=="RoadAccidents", :]
StopAndSearch = gdf.loc[gdf["Type"]=="StopAndSearch", :]
FireStations = gdf.loc[gdf["Type"]=="FireStations", :]
roads = gdf.loc[gdf["Type"]=='Road', :]
A_Road = roads.loc[roads["Name"]=='A Road']
Motorway = roads.loc[roads["Name"]=='Motorway']

National_Parks = gdf.loc[gdf["Type"]=="National_Parks", :]
GreenSpace = gdf.loc[gdf["Type"]=="GreenSpace", :]
Rivers = gdf.loc[gdf["Type"]=="Rivers", :]
Woodland_Region = gdf.loc[gdf["Type"]=="Woodland_Region", :]
Coastline = gdf.loc[gdf["Type"]=="Coastline", :]
Lakes = gdf.loc[gdf["Type"]=="Lakes", :]

TotalRainfall_mm_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='TotalRainfall_mm_2020'),:]
Snow_Days_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='Snow_Days_2020'),:]
Sunlight_h_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='Sunlight_h_2020'),:]
AverageTemperature_C_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='AverageTemperature_C_2020'),:]
GroundFrost_Days_2020 = gdf.loc[(gdf["Type"]=='MetOffice') & (gdf["Name"]=='GroundFrost_Days_2020'),:]
#...etc.

#Point datasets to find the density of and the radius of those densities in the format output_name:dataframe where the output variable will have the name Outname_Density_Radius
density_var_dict = {
    "Postcodes":Postcodes, 
    "All_Crimes":All_Crimes, 
    "All_Schools":All_Schools, 
    "Hospitals":Hospitals, 
    "RailwayStations":RailwayStations,
    "FerryTerminals":FerryTerminals,
    "BusStops":BusStops,
    "ListedBuildings":ListedBuildings,
    "RoadAccidents":RoadAccidents,
    "StopAndSearchs":StopAndSearch,
    "FireStations":FireStations
}

density_r_list = [1000,5000,25000]
#density_r_list = [1000]

#Point datasets to find the nearest of in the format output_name:dataframe where the output variable will have the name Nearest_Out_Name
nearest_var_dict = {
    "School":All_Schools, 
    "Hospital":Hospitals, 
    "RailwayStation":RailwayStations,
    "FerryTerminals":FerryTerminals,
    "BusStops":BusStops,
    "ListedBuilding":ListedBuildings,
    "A_Road":A_Road,
    "Motorway":Motorway,
    "National_Parks":National_Parks,
    "GreenSpace":GreenSpace,
    "Rivers":Rivers,
    "Woodland_Region":Woodland_Region,
    "Coastline":Coastline,
    "Lakes":Lakes
}

#Points to find the average of
average_var_dict = {
    "ProertyPrice_2020_2021":[PropertySales, "Details_Float"],
    "TotalRainfall_mm_2020":[TotalRainfall_mm_2020, "Details_Float"],
    "Snow_Days_2020":[Snow_Days_2020, "Details_Float"],
    "Sunlight_h_2020":[Sunlight_h_2020, "Details_Float"],
    "AverageTemperature_C_2020":[AverageTemperature_C_2020, "Details_Float"],
    "GroundFrost_Days_2020":[GroundFrost_Days_2020, "Details_Float"]
}

In [ ]:
#Start with adding density columns
for k in density_var_dict:
    print("Variable: " + k)
    for r in density_r_list:
        now = datetime.datetime.now()
        print("Density: " + str(r))
        print (now.strftime("%Y-%m-%d %H:%M:%S"))
        Postcodes[k + "_Density_" + str(r)] = kf.within_radius(Postcodes,density_var_dict[k], r)

#this step is the very slow one

In [ ]:
#Add distance to the nearest, don't include geometry lines
for k in nearest_var_dict:
    print(k)
    Postcodes[k + "_Distance_to_Nearest"] = kf.dist_to_nearest(Postcodes, nearest_var_dict[k], return_geom = False)


School
Hospital
RailwayStation
FerryTerminals
BusStops
ListedBuilding
A_Road
Motorway
National_Parks
GreenSpace
Rivers
Woodland_Region
Coastline
Lakes


In [ ]:
#Find the average value within the list of radii
for k in average_var_dict:
    for r in density_r_list:
        Postcodes["Average_" + k + "_" + str(r)] = kf.average_within_radius(Postcodes, average_var_dict[k][0], average_var_dict[k][1], r)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
Postcodes.columns

In [ ]:
Postcodes.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.ioff()

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,15))
plt.title("Point Tests")
gdf.loc[gdf["Type"]=="All_GB",:].plot(ax=ax, color='none', edgecolor='black', zorder=0)
gdf.loc[(gdf["Type"]=="Road"),:].plot(ax=ax, color='black', linewidth=0.5, zorder=1)
gdf.loc[(gdf["Type"]=="GreenSpace"),:].plot(ax=ax, color='green', zorder=2)
Postcodes.plot(column='GreenSpace_Distance_to_Nearest', ax=ax, legend=True, alpha=0.5, zorder=3)
plt.savefig("D:/GeoData/WorkingData/Images/Point_Test.png",dpi=300)
#plt.close(fig)
plt.show()
